In [1]:
pip install lxml

     |████████████████████████████████| 6.9 MB 11.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import re

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **three** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon). <br />
We met: <br />
10/11: 5-7pm <br />
10/20: 10-12pm <br />
10/21: 7-11pm <br />

___


## Step 1 - Scrape SpaceWeatherLive.com

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",timeout=10,headers=headers)
r.status_code

200

In [4]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")

In [5]:
table = soup.findAll("table") #find the table in the HTML

In [6]:
solar_flare_df = pd.read_html(str(table))

In [7]:
solfla_df = solar_flare_df[0]#works when we just select the first table

In [8]:
solfla_df.columns = ["rank", "x_class", "date", "region", "start_time", "maximum_time", "end_time", "movie"] #rename columns to be more descriptive

In [9]:
solfla_df.head()

,rank,x_class,date,region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


___
## Step 2 - Tidy the Top 50 Solar Flare Data

In [10]:
solfla_df = solfla_df[["rank", "x_class", "date", "start_time", "maximum_time", "end_time","region"]]
solfla_df.head()

,rank,x_class,date,start_time,maximum_time,end_time,region
0,1,X28+,2003/11/04,19:29,19:53,20:06,486
1,2,X20+,2001/04/02,21:32,21:51,22:03,9393
2,3,X17.2+,2003/10/28,09:51,11:10,11:24,486
3,4,X17+,2005/09/07,17:17,17:40,18:03,808
4,5,X14.4,2001/04/15,13:19,13:50,13:55,9415


In [11]:
import datetime

In [12]:

solfla_df.loc[:,"start_time"] = pd.to_datetime(solfla_df.loc[:,"date"] + " " + solfla_df.loc[:,"start_time"])
solfla_df.loc[:,"maximum_time"] = pd.to_datetime(solfla_df.loc[:,"date"] + " " + solfla_df.loc[:,"maximum_time"])
solfla_df.loc[:,"end_time"] = pd.to_datetime(solfla_df.loc[:,"date"] + " " + solfla_df.loc[:,"end_time"])


/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [13]:
solfla_df.drop(columns="date",inplace=True) #drop date table
solfla_df.rename(columns={"start_time":"start_datetime", "end_time":"end_datetime","maximum_time":"maximum_datetime"},inplace=True)
solfla_df.head()

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,rank,x_class,start_datetime,maximum_datetime,end_datetime,region
0,1,X28+,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20+,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2+,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17+,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415


In [14]:
def remove_plus(string):
  return re.sub("\+", "", string,1)

def make_float(string): #make all number values in rating into floats to make searching easier later
  rating = string[0]
  string = string[1:]
  new_float = float(string)
  return rating + str(new_float)

  
solfla_df["x_class"] = solfla_df["x_class"].apply(remove_plus)
solfla_df["x_class"] = solfla_df["x_class"].apply(make_float)



/tmp/ipykernel_65/2216264708.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solfla_df["x_class"] = solfla_df["x_class"].apply(remove_plus)
/tmp/ipykernel_65/2216264708.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  solfla_df["x_class"] = solfla_df["x_class"].apply(make_float)


In [15]:
solfla_df.dtypes

rank                         int64
x_class                     object
start_datetime      datetime64[ns]
maximum_datetime    datetime64[ns]
end_datetime        datetime64[ns]
region                       int64
dtype: object

___
## Step 3 - Scrape the NASA Data

In [16]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",timeout=10,headers=headers)
r.status_code

200

In [17]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")
table = soup.get_text()
split_table = table.splitlines()
split_table = split_table[15:]
split_table = split_table[0:len(split_table)-2] #clean up table by removing top and bottom portion


In [18]:
nasa_df = pd.DataFrame( #create empty DF 
    columns = ["start_date","start_time","end_date","end_time","start_freq","end_freq","location","region","xray_importance","cme_date","cme_time","central_pos_angle","cpa_width","cme_speed","PHTX" ]
    )
for i,v in enumerate(split_table): 
  row_vals = re.findall(r"\S+",v)
  nasa_df.loc[i] = row_vals[0:15] #assign values for each row from list after splitting string into a list var, also drop 16th entry because sometimes it is useless notation

In [19]:
nasa_df.drop(columns=["PHTX"],inplace=True) #drop links column

In [20]:
nasa_df.head()

,start_date,start_time,end_date,end_time,start_freq,end_freq,location,region,xray_importance,cme_date,cme_time,central_pos_angle,cpa_width,cme_speed
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,74,79,312
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,Halo,360,878
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,Halo,360,464
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,263,165,296
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,133,155,712


___
## Step 4 - Tidy the Nasa Data

**Replace all empty values with NaNs**

In [21]:
nasa_df.region.replace("-----", np.nan,inplace=True)
nasa_df.xray_importance.replace("----", np.nan,inplace=True)
nasa_df.cme_date.replace("--/--",np.nan, inplace=True)
nasa_df.cme_time.replace("--:--",np.nan,inplace=True)
nasa_df.central_pos_angle.replace("----", np.nan,inplace=True)
nasa_df.cpa_width.replace("----", np.nan,inplace=True)
nasa_df.cpa_width.replace("---", np.nan,inplace=True)
nasa_df.cpa_width.replace("360h", "360",inplace=True)
nasa_df.cme_speed.replace("----", np.nan,inplace=True)
nasa_df.start_freq.replace("????",np.nan,inplace=True)
nasa_df.end_freq.replace("????",np.nan,inplace=True)



**Replace all lower case "Back" values with upper case "BACK" in location column**

In [22]:
nasa_df.location.replace("Back","BACK",inplace=True)

**Replace all DSF values with FILA in region column**

In [23]:
nasa_df.region.replace("DSF","FILE",inplace=True)

**Create column to indicate HALO flare, then turn Halo entries into NaN.**<br/>
Halo = 1 <br/>
Non-Halo = 0

In [24]:
column_vals = []
for index,value in nasa_df["central_pos_angle"].iteritems():
  if value == "Halo":
    column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["is_halo"] = column_vals

In [25]:
nasa_df.central_pos_angle.replace("Halo", np.nan,inplace=True)

**Indicate which cpa_widths are lower bounds, make a new column with the info, then remove the non-numeric characters**

In [26]:
column_vals = []
for index,value in nasa_df["cpa_width"].iteritems():
  if pd.isnull(value) == True:
    column_vals.append(0)
  elif ">" in value:
        column_vals.append(1)
  else:
    column_vals.append(0)
nasa_df["lower_bound"] = column_vals

In [27]:
def remove_sign(str):
  if pd.isnull(str) == True:
    return str
  return re.sub(">","",str)

nasa_df["cpa_width"] = nasa_df["cpa_width"].apply(remove_sign)

**Reformat date columns and drop repetitive data**

In [28]:
nasa_df.replace("24:00","23:59",inplace=True)

In [29]:
for index,value in nasa_df["end_date"].iteritems():
  year = nasa_df.start_date.loc[index][0:4]
  nasa_df["end_date"].loc[index] = pd.to_datetime(year + "/" + nasa_df["end_date"].loc[index] + " " + nasa_df["end_time"].loc[index])

for index,value in nasa_df["cme_date"].iteritems():
  year = nasa_df.start_date.loc[index][0:4]
  if pd.isnull(nasa_df["cme_date"].loc[index]):
    pass
  else:
    nasa_df["cme_date"].loc[index] = pd.to_datetime(year + "/" + nasa_df["cme_date"].loc[index] + " " + nasa_df["cme_time"].loc[index])

nasa_df.loc[:,"start_date"] = pd.to_datetime(nasa_df.loc[:,"start_date"] + " " + nasa_df.loc[:,"start_time"])
nasa_df.loc[:,"end_date"] = pd.to_datetime(nasa_df.loc[:,"end_date"])
nasa_df.loc[:,"cme_date"] = pd.to_datetime(nasa_df.loc[:,"cme_date"])


/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [30]:
nasa_df.rename(columns={"start_date":"start_datetime", "end_date":"end_datetime","cme_date":"cme_datetime"},inplace=True) #rename columns

In [31]:
nasa_df.drop(columns=["start_time","end_time","cme_time"],inplace=True) #drop unnessecary values

In [32]:
nasa_df.start_freq = nasa_df.start_freq.astype('float') #change all str types to float
nasa_df.end_freq = nasa_df.end_freq.astype('float')
nasa_df.central_pos_angle = nasa_df.central_pos_angle.astype('float')
nasa_df.cme_speed = nasa_df.cme_speed.astype('float')
nasa_df.cpa_width = nasa_df.cpa_width.astype('float')

**Adjust xray_importance column to have 0 after "." value for easier searching later**

In [33]:
nasa_df.dtypes

start_datetime       datetime64[ns]
end_datetime         datetime64[ns]
start_freq                  float64
end_freq                    float64
location                     object
region                       object
xray_importance              object
cme_datetime         datetime64[ns]
central_pos_angle           float64
cpa_width                   float64
cme_speed                   float64
is_halo                       int64
lower_bound                   int64
dtype: object

In [34]:
nasa_df.head()

,start_datetime,end_datetime,start_freq,end_freq,location,region,xray_importance,cme_datetime,central_pos_angle,cpa_width,cme_speed,is_halo,lower_bound
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000.0,4000.0,S25E16,8026,M1.3,1997-04-01 15:18:00,74.0,79.0,312.0,0,0
1,1997-04-07 14:30:00,1997-04-07 17:30:00,11000.0,1000.0,S28E19,8027,C6.8,1997-04-07 14:27:00,NaN,360.0,878.0,1,0
2,1997-05-12 05:15:00,1997-05-14 16:00:00,12000.0,80.0,N21W08,8038,C1.3,1997-05-12 05:30:00,NaN,360.0,464.0,1,0
3,1997-05-21 20:20:00,1997-05-21 22:00:00,5000.0,500.0,N05W12,8040,M1.3,1997-05-21 21:00:00,263.0,165.0,296.0,0,0
4,1997-09-23 21:53:00,1997-09-23 22:16:00,6000.0,2000.0,S29E25,8088,C1.4,1997-09-23 22:02:00,133.0,155.0,712.0,0,0


___
## Part 2 - Analysis


###Question 1: Replication
Can you replicate the top 50 solar flare table in SpaceWeatherLive.com exactly using the data obtained from NASA? That is, if you get the top 50 solar flares from the NASA table based on their classification (e.g., X28 is the highest), do you get data for the same solar flare events?

Include code used to get the top 50 solar flares from the NASA table (be careful when ordering by classification, remember x is the highest!). Write a sentence or two discussing how well you can replicate the SpaceWeatherLive data from the NASA data.

In [35]:
display(nasa_df.head())
display(solfla_df.head())

,start_datetime,end_datetime,start_freq,end_freq,location,region,xray_importance,cme_datetime,central_pos_angle,cpa_width,cme_speed,is_halo,lower_bound
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000.0,4000.0,S25E16,8026,M1.3,1997-04-01 15:18:00,74.0,79.0,312.0,0,0
1,1997-04-07 14:30:00,1997-04-07 17:30:00,11000.0,1000.0,S28E19,8027,C6.8,1997-04-07 14:27:00,NaN,360.0,878.0,1,0
2,1997-05-12 05:15:00,1997-05-14 16:00:00,12000.0,80.0,N21W08,8038,C1.3,1997-05-12 05:30:00,NaN,360.0,464.0,1,0
3,1997-05-21 20:20:00,1997-05-21 22:00:00,5000.0,500.0,N05W12,8040,M1.3,1997-05-21 21:00:00,263.0,165.0,296.0,0,0
4,1997-09-23 21:53:00,1997-09-23 22:16:00,6000.0,2000.0,S29E25,8088,C1.4,1997-09-23 22:02:00,133.0,155.0,712.0,0,0


,rank,x_class,start_datetime,maximum_datetime,end_datetime,region
0,1,X28.0,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00,486
1,2,X20.0,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00,9393
2,3,X17.2,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00,486
3,4,X17.0,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00,808
4,5,X14.4,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00,9415


**Slightly modify Nasa table for easier searching (add zero after decimal points with no trailing ints i.e. 28. -> 28.0** 

In [36]:
def add_zero(entry):
  if (pd.isnull(entry) == True):
    return entry
  if (entry[len(entry)-1] == "."):
    return entry + "0"
  else:
    return entry

nasa_df["xray_importance"] = nasa_df["xray_importance"].apply(add_zero)

###Sorting values of Nasa Table xray_importance column to replicate Solar Flare DataFrame

In [37]:
class_no_nan = [x for x in nasa_df.xray_importance.sort_values(ascending=False) if pd.isnull(x) == False]
class_no_nan = class_no_nan[0:50]

def strip_and_sort(lst):
  rm_x_lst = []
  for value in lst:
    rm_x_lst.append(float(value[1:]))
  rm_x_lst.sort(reverse=True)
  return [("X" + str(flt_val)) for flt_val in rm_x_lst]

sorted_flares = strip_and_sort(class_no_nan)

**Instantiate replica dataframe**

In [38]:
imposter_solfla_df = pd.DataFrame(
    columns = ["rank", "x_class","start_datetime", "end_datetime","region"]
)

**Fill replace DF with values from NASA table**

In [39]:
# for index,value in solfla_df.loc[:,["x_class","start_datetime"]].iterrows():
#     x_class, start_date = value[0], value[1]
#     for nasa_index,nasa_value in nasa_df.loc[:,["xray_importance","start_datetime"]].iterrows():
#       if (x_class == nasa_value[0]):
#         if (start_date.year == nasa_value[1].year) and (start_date.month == nasa_value[1].month) and (start_date.day == nasa_value[1].day):
#           imposter_solfla_df.loc[index] = [index+1, x_class, nasa_df["start_datetime"].loc[nasa_index], nasa_df["end_datetime"].loc[nasa_index], nasa_df["region"].loc[nasa_index]]

In [40]:
for x_class in sorted_flares:
  start_date = solfla_df[solfla_df.x_class == x_class].start_datetime
  for date in start_date:
    for nasa_index,nasa_value in nasa_df.loc[:,["xray_importance","start_datetime"]].iterrows():
      if (x_class == nasa_value[0]):
        if (date.year == nasa_value[1].year) and (date.month == nasa_value[1].month) and (date.day == nasa_value[1].day):
          imposter_solfla_df.loc[index] = [index+1, x_class, nasa_df["start_datetime"].loc[nasa_index], nasa_df["end_datetime"].loc[nasa_index], nasa_df["region"].loc[nasa_index]]



In [41]:
display(len(imposter_solfla_df)) #FIXME Problem with some entries being skipped, and others being duplicated. Not sure of the issue right now.

1

**Ans**: No, you cannot exactly replicate the SpaceWeatherLive.com table using the Nasa data. Some of the Nasa data is inconsistent with the Solar Flare data (off by 0.1 for example). Not to mention the fact that Nasa does not have the maximum time data either.
___

In [42]:
imposter_solfla_df

,rank,x_class,start_datetime,end_datetime,region
521,522,X2.6,2001-09-24 10:45:00,2001-09-25 20:00:00,9632


### Question 2 - Ranking
Write a function that finds the best matching row in the NASA data for each of the top 50 solar flares in the SpaceWeatherLive data and assigns a rank value to that entry in the NASA table. Here, you have to decide for yourself how you determine what is the best matching entry in the NASA data for each of the top 50 solar flares.

Hint: You may want to look at multiple pieces of data before deciding if a flare matches.

In your submission, include an explanation of how you are defining best matching rows across the two datasets in addition to the code used to find the best matches.

Use your function to add a new column to the NASA dataset indicating its rank according to SpaceWeatherLive, if it appears in that dataset.

Hint: You may not end up with all the ranks!

At the end of this section display only flares in the NASA table that you could rank.

In [96]:
def rank_data(solfla_df, nasa_df):
    solfla_rank = []
    for index, row in solfla_df.iterrows():
        for index1, row1 in nasa_df.iterrows():
            if ((row["start_datetime"] == row1["start_datetime"]) & (row["end_datetime"] == row1["end_datetime"])):
                solfla_rank += row["rank"]
    nasa_df["solfla_rank"] = solfla_rank
            

nasa_df 

,start_datetime,end_datetime,start_freq,end_freq,location,region,xray_importance,cme_datetime,central_pos_angle,cpa_width,cme_speed,is_halo,lower_bound,solfla_rank
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000.0,4000.0,S25E16,8026,M1.3,1997-04-01 15:18:00,74.0,79.0,312.0,0,0,1.0
1,1997-04-07 14:30:00,1997-04-07 17:30:00,11000.0,1000.0,S28E19,8027,C6.8,1997-04-07 14:27:00,NaN,360.0,878.0,1,0,2.0
2,1997-05-12 05:15:00,1997-05-14 16:00:00,12000.0,80.0,N21W08,8038,C1.3,1997-05-12 05:30:00,NaN,360.0,464.0,1,0,3.0
3,1997-05-21 20:20:00,1997-05-21 22:00:00,5000.0,500.0,N05W12,8040,M1.3,1997-05-21 21:00:00,263.0,165.0,296.0,0,0,4.0
4,1997-09-23 21:53:00,1997-09-23 22:16:00,6000.0,2000.0,S29E25,8088,C1.4,1997-09-23 22:02:00,133.0,155.0,712.0,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2017-09-17 11:45:00,2017-09-17 12:35:00,16000.0,900.0,S08E170,NaN,NaN,2017-09-17 12:00:00,NaN,360.0,1385.0,1,0,NaN
518,2017-10-18 05:48:00,2017-10-18 12:40:00,16000.0,400.0,S06E123,NaN,NaN,2017-10-18 08:00:00,85.0,146.0,1001.0,0,0,NaN
519,2019-05-03 23:52:00,2019-05-04 00:16:00,13000.0,2300.0,N12E82,12740,C1.0,2019-05-03 23:24:00,90.0,113.0,692.0,0,0,NaN
520,2020-11-29 13:07:00,2020-11-29 15:23:00,14000.0,850.0,S23E89,NaN,M4.4,2020-11-29 13:25:00,NaN,360.0,2077.0,1,0,NaN
